In [10]:
# Make sure web3 is installed-
# pip install web3

In [9]:
from web3 import Web3
import json

# Connect to Ethereum mainnet
w3 = Web3(Web3.HTTPProvider('https://eth.drpc.org'))

# Check connection
if w3.is_connected():
    print("Connected to Ethereum mainnet.")
else:
    print("Connection failed.")
    exit()

# Vault contract address and ABI
vault_address = '0xBA12222222228d8Ba445958a75a0704d566BF2C8'
with open('vault.json') as f:
    vault_abi = json.load(f)
vault_contract = w3.eth.contract(address=vault_address, abi=vault_abi)

# Token addresses for USDC, DAI, and WETH
TOKEN_ADDRESSES = {
    "USDC": '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48',
    "DAI": '0x6B175474E89094C44Da98b954EedeAC495271d0F',
    "WETH": '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'
}

# Pool IDs in bytes32 format
usdc_weth_pool_id = bytes.fromhex('96646936b91d6b9d7d0c47c496afbf3d6ec7b6f8000200000000000000000019')
weth_dai_pool_id = bytes.fromhex('0b09dea16768f0799065c475be02919503cb2a3500020000000000000000001a')
usdc_dai_pool_id = bytes.fromhex('79c58f70905f734641735bc61e45c19dd9ad60bc0000000000000000000004e7')

# Function to get tokens and balances in a specified pool
def get_pool_tokens(pool_id):
    try:
        tokens, balances, last_change_block = vault_contract.functions.getPoolTokens(pool_id).call()
        token_balance_mapping = {token: balance for token, balance in zip(tokens, balances)}

        # Display tokens and their respective balances
        print(f"Tokens in Pool {pool_id.hex()}:")
        for token, balance in token_balance_mapping.items():
            print(f"Token Address: {token}, Balance: {balance}")

        return token_balance_mapping
    except Exception as e:
        print("Error fetching pool tokens:", e)
        return {}

# Function to get token decimals
def get_token_decimals(token_address):
    erc20_abi = '[{"constant":true,"inputs":[],"name":"decimals","outputs":[{"name":"","type":"uint8"}],"type":"function"}]'
    token_contract = w3.eth.contract(address=token_address, abi=json.loads(erc20_abi))
    return token_contract.functions.decimals().call()

# Function to calculate and print price between two tokens
def calculate_price(pool_id, token0, token1):
    # Retrieve token balances for the pool
    token_balance_mapping = get_pool_tokens(pool_id)

    # Check if both tokens are in the pool
    if token0 in token_balance_mapping and token1 in token_balance_mapping:
        # Get balances and decimals for each token
        balance0 = token_balance_mapping[token0]
        balance1 = token_balance_mapping[token1]
        decimals0 = get_token_decimals(token0)
        decimals1 = get_token_decimals(token1)

        # Convert balances to standard units
        balance0_in_standard_units = balance0 / (10 ** decimals0)
        balance1_in_standard_units = balance1 / (10 ** decimals1)

        # Calculate price of token0 in terms of token1
        if balance1_in_standard_units != 0:
            price = balance0_in_standard_units / balance1_in_standard_units
            print(f"Price of {token0} in terms of {token1}: {price}")
            return price
        else:
            print("Insufficient balance to calculate price.")
            return None
    else:
        print(f"Tokens {token0} and {token1} not found in pool {pool_id.hex()}")
        return None

# Calculate prices for each pair and display results
print("USDC/WETH Pool on Ethereum:")
usdc_weth_price = calculate_price(usdc_weth_pool_id, TOKEN_ADDRESSES["USDC"], TOKEN_ADDRESSES["WETH"])

print("\nWETH/DAI Pool on Ethereum:")
weth_dai_price = calculate_price(weth_dai_pool_id, TOKEN_ADDRESSES["WETH"], TOKEN_ADDRESSES["DAI"])

print("\nUSDC/DAI Pool on Ethereum:")
usdc_dai_price = calculate_price(usdc_dai_pool_id, TOKEN_ADDRESSES["USDC"], TOKEN_ADDRESSES["DAI"])

# Calculate arbitrage product if all prices are retrieved successfully
if usdc_weth_price and weth_dai_price and usdc_dai_price:
    arbitrage_product = usdc_weth_price * weth_dai_price * (1 / usdc_dai_price)
    print(f"\nArbitrage Product: {arbitrage_product}")

    # Check for potential arbitrage
    if arbitrage_product > 1:
        print("Arbitrage opportunity detected!")
    else:
        print("No arbitrage opportunity detected.")
else:
    print("Could not retrieve all prices. Skipping arbitrage calculation.")


Connected to Ethereum mainnet.
USDC/WETH Pool on Ethereum:
Tokens in Pool 96646936b91d6b9d7d0c47c496afbf3d6ec7b6f8000200000000000000000019:
Token Address: 0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48, Balance: 131072486598
Token Address: 0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2, Balance: 54530999877140396826
Price of 0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48 in terms of 0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2: 2403.6325556712573

WETH/DAI Pool on Ethereum:
Tokens in Pool 0b09dea16768f0799065c475be02919503cb2a3500020000000000000000001a:
Token Address: 0x6B175474E89094C44Da98b954EedeAC495271d0F, Balance: 94923373763718172209271
Token Address: 0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2, Balance: 59250602876999737103
Price of 0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2 in terms of 0x6B175474E89094C44Da98b954EedeAC495271d0F: 0.0006241940264837767

USDC/DAI Pool on Ethereum:
Tokens in Pool 79c58f70905f734641735bc61e45c19dd9ad60bc0000000000000000000004e7:
Token Address: 0x6B175474E89094C44